# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [8]:
import mindspore as ms
from mindspore import nn, ops
import numpy as np
import troubleshooter as ts

x = ops.arange(4)
ts.save('x-file', x)

我们现在可以将存储在文件中的数据读回内存。


In [9]:
x2 = np.load('0_x-file.npy')
x2

array([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]


In [10]:
y = ops.zeros(4)
ts.save('x-files', [x, y])

In [11]:
x2, y2 = np.load('3_x-files_0.npy'),  np.load('3_x-files_1.npy'),  
(x2, y2)

(array([0, 1, 2, 3]), array([0., 0., 0., 0.], dtype=float32))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [12]:
mydict = {'x': x, 'y': y}
ts.save('mydict', mydict)

In [13]:
mydict2 = {'x': np.load('4_mydict_x.npy'), 'y': np.load('4_mydict_y.npy')}
mydict2

{'x': array([0, 1, 2, 3]), 'y': array([0., 0., 0., 0.], dtype=float32)}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。


In [15]:
class MLP(nn.Cell):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Dense(20, 256)
        self.output = nn.Dense(256, 10)

    def construct(self, x):
        return self.output(ops.relu(self.hidden(x)))

net = MLP()
X = ops.randn((2, 20))
Y = net(X)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]


In [16]:
ms.save_checkpoint(net, 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [18]:
clone = MLP()
ms.load_checkpoint('mlp.params.ckpt', clone)
clone.set_train(False)

MLP<
  (hidden): Dense<input_channels=20, output_channels=256, has_bias=True>
  (output): Dense<input_channels=256, output_channels=10, has_bias=True>
  >

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [19]:
Y_clone = clone(X)
Y_clone == Y

Tensor(shape=[2, 10], dtype=Bool, value=
[[ True,  True,  True ...  True,  True,  True],
 [ True,  True,  True ...  True,  True,  True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？


[Discussions](https://discuss.d2l.ai/t/1839)
